In [4]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [26]:
## Environment
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/models'):
    main_path = p[:-len('/models')]
sys.path[0] = main_path

import os, gc, json 
from termcolor import colored
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from src import (config, fe, features, feature_selection, 
                 preprocess, training)
from src.fe import CreateFeatures

In [6]:
# DATA PREPARATION
# ===== LOAD DATA ======
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
print(f'Metadata: {metadata.shape}')

train_labels = pd.read_csv(config.DATA_DIR + 'train_labels.csv')
print(f'Train labels: {train_labels.shape}')

valid_labels = pd.read_csv(config.DATA_DIR + 'val_labels.csv')
print(f'Train labels: {valid_labels.shape}')

# Combine train and valid labels
trvl_labels = pd.concat([train_labels, valid_labels], axis = 0)

submission = pd.read_csv(config.DATA_DIR + 'submission_format.csv')
print(f'Submission: {submission.shape}')

# ===== FILE PATHS OF SAMPLES =====
train_files = metadata[metadata.split == 'train']['features_path'].to_dict()
valid_files = metadata[metadata.split == 'val']['features_path'].to_dict()
test_files = metadata[metadata.split == 'test']['features_path'].to_dict()
# Train & Valid files
trva_files = train_files.copy()
trva_files.update(valid_files)
# All files
all_test_files = valid_files.copy()
all_test_files.update(test_files)

# Define SAM testbed files
sam_files = metadata[(metadata.instrument_type == 'sam_testbed') & (metadata.split == 'train')]['features_path']
sam_files = sam_files.to_dict()

# Get the names of the target columns in a list
target_labels_list = [i for i in train_labels.columns if i not in ['sample_id']]
print(target_labels_list)

# SAM testbed labels
sam_labels = train_labels.drop(train_labels.tail(len(sam_files)).index)
sam_labels = pd.concat([sam_labels, valid_labels], axis=0)
print(f'Labels w/o SAM : {sam_labels.shape}')

Metadata: (1570, 5)
Train labels: (766, 11)
Train labels: (293, 11)
Submission: (804, 11)
['basalt', 'carbonate', 'chloride', 'iron_oxide', 'oxalate', 'oxychlorine', 'phyllosilicate', 'silicate', 'sulfate', 'sulfide']
Labels w/o SAM : (1047, 11)


Define the global variables:

In [43]:
LABEL = 'basalt'
# Base model features
INITIAL_FTS = 'fts_mra_tempmz'
# Features to add to the base model
ALL_FTS_GROUPS = config.FTS_GROUPS
MODEL_ALGO = 'XGB_opt'

In [64]:
SPLIT_TYPE = 'tr'

# Save best model features: CV loss the lowest when compared to previous fitted model
BEST_MODEL_FTS_GROUPS = [INITIAL_FTS]   # Name of a feature group included in the final model
BEST_MODEL_FTS = []                     # Selected features after SFM if applied
BEST_MODEL_CVLOSS = 0
BEST_MODEL_VLOSS = 0

**INITIAL TRAINING DATA**

In [65]:
# Inital X_train: base model features
X_train = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 
                                   INITIAL_FTS + '_' + SPLIT_TYPE + '.csv'))
print(f'X_train initial: {X_train.shape}')
initial_fts_path = INITIAL_FTS + '_' + MODEL_ALGO + '_' + SPLIT_TYPE + '_SFM_COLS.txt'
if os.path.exists(initial_fts_path):
    print(f'Reading features:', colored(f'{initial_fts_path}', 'yellow'))
    with open(initial_fts_path) as json_file:
            initial_fts_dict = json.load(json_file)
    BEST_MODEL_FTS = BEST_MODEL_FTS + initial_fts_dict[LABEL]
    print(f'No features added:',colored(f'{len(initial_fts_dict[LABEL])}', 'blue'))
else:
    BEST_MODEL_FTS = BEST_MODEL_FTS + X_train.columns.tolist()
    print(f'No features added:',colored(f'{len(X_train.columns.tolist())}', 'blue'))

# Redefine X_train given new set of features
X_train = X_train[BEST_MODEL_FTS].copy()
print(colored(f'X_train base: {X_train.shape}', 'green'))

# Define validation & test dataset
X_vlte = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 
                                  INITIAL_FTS + '_vlte.csv'))
X_vlte = X_vlte[BEST_MODEL_FTS].copy()
print(colored(f'X_vlte base: {X_vlte.shape}', 'green'))

X_train initial: (766, 1584)
Reading features: fts_mra_tempmz_XGB_opt_tr_SFM_COLS.txt
No features added: 31
X_train base: (766, 31)
X_vlte base: (804, 31)


In [42]:
for FTS_GROUP in ALL_FTS_GROUPS[:2]:
    print(f'\nBase features: {BEST_MODEL_FTS_GROUPS}')
    print(f'Feature group:', colored(f'{FTS_GROUP}', 'yellow'))

    # ===== DATASET =====
    X_train = pd.DataFrame()
    
    # If the feature is not already included in the best model
    if FTS_GROUP not in BEST_MODEL_FTS_GROUPS:
        # Combine BEST_MODEL_FTS_GROUPS and FTS_GROUP
        for fg in BEST_MODEL_FTS_GROUPS:
            df_fg = pd.read_csv(os.path.join(config.DATA_DIR_OUT, fg + '_' + SPLIT_TYPE + '.csv'))
            X_train = pd.concat([X_train, df_fg], axis=1)

            # Get columns
            df_fg_cols_path = fg + '_' + MODEL_ALGO + '_' + SPLIT_TYPE + '_SFM_COLS.txt'     
            if os.path.exists(df_fg_cols_path):
                print(f'Reading features: {df_fg_cols_path}')
                with open(df_fg_cols_path) as json_file:
                        fg_fts_dict = json.load(json_file)
                BEST_MODEL_FTS.append(fg_fts_dict[LABEL])
            else:
                print(f'Appending features: {df_fg_cols_path}')
                BEST_MODEL_FTS = BEST_MODEL_FTS + df_fg.columns.tolist()
            
        # Add current feature group
        df_cf = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_GROUP + '_' + SPLIT_TYPE + '.csv'))
        X_train = pd.concat([X_train, df_cf], axis=1)
        print(f'X_train: {X_train.shape}')

        # Combine BEST_MODEL_FTS and FTS_GROUP columns
        
        
        # Train & compare logloss
        
        # Record stats


Base features: ['fts_mra_tempmz']
Feature group: fts_topmz
Reading features: fts_mra_tempmz_XGB_opt_tr_SFM_COLS.txt
X_train: (766, 1587)

Base features: ['fts_mra_tempmz']
Feature group: fts_slope_tt
Reading features: fts_mra_tempmz_XGB_opt_tr_SFM_COLS.txt
X_train: (766, 1585)


In [39]:
BEST_MODEL_FTS

[]

In [6]:
FTS_NAME = 'fts_mra_tempmz_maxabun'                 # Name of the file with base features for TRAINING
COMPUTE_FTS = True                                      # Should the features be recomputed
COMPUTE_FTS_SAM = False                                 # Compute SAM test bed
MODEL_ALGO = 'XGB_opt'                                  # Name of the classifier
MODEL_NAME = FTS_NAME + '_' + MODEL_ALGO                # Name of the model
COMBINE_FTS = ['fts_mra_tempmz', 'fts_mz_maxabun']   # Feature sets to combine for training
NEW_FEATURES = None                                        # Name of a data frame with new features to add to model
TRAIN_FTS_SFM = ['fts_mra_tempmz', 'fts_mz_maxabun'] # Features selected with SMF() for training.   
BASE_MODEL_FTS = 'fts_mra_tempmz'
BASE_MODEL = BASE_MODEL_FTS + '_' + MODEL_ALGO

**COMPUTE FEATURES**

- Change the `fe._` method depending on the feature that we wish to calculate

In [7]:
# Check if feature is computed and load it or choose to compute it
check_file = 0
for i in ['_tr', '_trvl', '_vlte']:
    check_file += os.path.exists(os.path.join(config.DATA_DIR_OUT, FTS_NAME +
                                              str(i) + '.csv'))

if (check_file == 3) & (not COMPUTE_FTS):
    print('Reading features ... ')
    X_tr = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_tr.csv'))
    print(X_tr.shape)
    X_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_trvl.csv'))
    print(X_trvl.shape)
    X_vlte = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_vlte.csv'))
    print(X_vlte.shape)
    
else:
    print('Computing features ... ')
    # ----- TRAIN -----
    fe = CreateFeatures(metadata, train_files, 'tr', FTS_NAME)
    X_tr = fe.combine_features(COMBINE_FTS)
    print(colored(f'train => {X_tr.shape}', 'blue'))
    
    # ----- TRAIN & VALID -----
    fe = CreateFeatures(metadata, trva_files, 'trvl', FTS_NAME)
    X_trvl = fe.combine_features(COMBINE_FTS)
    print(colored(f'train & valid => {X_trvl.shape}', 'blue'))
    
    # ----- VALID & TEST -----
    fe = CreateFeatures(metadata, all_test_files, 'vlte', FTS_NAME)
    X_vlte = fe.combine_features(COMBINE_FTS)
    print(colored(f'valid & test => {X_vlte.shape}', 'blue'))
    
if COMPUTE_FTS_SAM:
    print(f'\nCreating SAM testbed samples ...')
    # Training without SAM testbed
    X_tr_sam = X_tr.drop(X_tr.tail(len(sam_files)).index).copy()
    X_tr_sam = pd.concat([X_tr_sam, X_trvl.iloc[len(train_files):,:]], axis=0)
    print(f'Train shape: {X_tr_sam.shape}')
    # Validation data
    X_vl_sam = X_tr.tail(len(sam_files)).copy()
    print(f'Valid shape: {X_vl_sam.shape}')

Computing features ... 
train => (766, 1683)
train & valid => (1059, 1683)
valid & test => (804, 1683)


## MODELS

**TRAIN**

In [8]:
split_type = 'tr'
feature_selection.combine_sfm_features(
                    base_sfm_features_name=TRAIN_FTS_SFM,
                    fitted_model_algo=MODEL_ALGO,
                    target_labels_list=target_labels_list,
                    split_type=split_type,
                    fitted_model_name=FTS_NAME)
#TRAIN_FTS_SFM = FTS_NAME

Adding fts_mra_tempmz
Adding fts_mz_maxabun
Saving fts_mra_tempmz_maxabun_XGB_opt_tr_SFM_COLS.txt


In [9]:
# ===== TRAIN =====
split_type = 'tr'

# Initialize the feature selection class
smf = feature_selection.SelectModelFeatures(
    base_sfm_features_name=TRAIN_FTS_SFM,
    base_fitted_model_name=None,
    target_labels_list=target_labels_list,
    new_features_file_name=NEW_FEATURES,
    fitted_model_name=MODEL_NAME,
    fitted_model_algo=MODEL_ALGO,
    X_tr=X_tr,
    X_vlte=X_vlte,
    split_type='tr',
    train_labels=train_labels,
    valid_files=valid_files,
    valid_labels=valid_labels)

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features(file_name=FTS_NAME)
else: 
    TRAIN_FTS_DICT = None

# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_tr,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    base_model_name=BASE_MODEL_FTS + '_' + MODEL_ALGO + '_' + split_type + '_sfm',
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

Loading feature column names
Reading fts_mra_tempmz_maxabun_XGB_opt_tr_SFM_COLS.txt
CV training ....
Basel model CVloss: ../models/fts_mra_tempmz_XGB_opt_tr_sfm_cvloss.csv
basalt: LogLoss=0.19076 -> -0.01303
carbonate: LogLoss=0.10985 -> 0.00105
chloride: LogLoss=0.18517 -> -0.02477
iron_oxide: LogLoss=0.22697 -> 0.00276
oxalate: LogLoss=0.02271 -> -0.00327
oxychlorine: LogLoss=0.1644 -> -0.01577
phyllosilicate: LogLoss=0.24467 -> -0.0007
silicate: LogLoss=0.24195 -> 0.00877
sulfate: LogLoss=0.20801 -> 0.01785
sulfide: LogLoss=0.06749 -> -4e-05
Full training .....
basalt - nfeatures: 55
carbonate - nfeatures: 143
chloride - nfeatures: 91
iron_oxide - nfeatures: 249
oxalate - nfeatures: 27
oxychlorine - nfeatures: 83
phyllosilicate - nfeatures: 316
silicate - nfeatures: 1604
sulfate - nfeatures: 174
sulfide - nfeatures: 123
Saving fts_mra_tempmz_maxabun_XGB_opt_tr_COLS_sfm.txt
CV LogLoss: 0.1662
VAL LogLoss: 0.15337


**TRAIN & VALID**

In [10]:
FTS_NAME = 'fts_mra_tempmz_maxabun'                 # Name of the file with base features for TRAINING
COMPUTE_FTS = True                                      # Should the features be recomputed
COMPUTE_FTS_SAM = False                                 # Compute SAM test bed
MODEL_ALGO = 'XGB_opt'                                  # Name of the classifier
MODEL_NAME = FTS_NAME + '_' + MODEL_ALGO                # Name of the model
COMBINE_FTS = ['fts_mra_tempmz', 'fts_mz_maxabun']   # Feature sets to combine for training
NEW_FEATURES = None                                        # Name of a data frame with new features to add to model
TRAIN_FTS_SFM = ['fts_mra_tempmz', 'fts_mz_maxabun'] # Features selected with SMF() for training.   
BASE_MODEL_FTS = 'fts_mra_tempmz'
BASE_MODEL = BASE_MODEL_FTS + '_' + MODEL_ALGO

In [11]:
split_type = 'trvl'
feature_selection.combine_sfm_features(
                    base_sfm_features_name=TRAIN_FTS_SFM,
                    fitted_model_algo=MODEL_ALGO,
                    target_labels_list=target_labels_list,
                    split_type=split_type,
                    fitted_model_name=FTS_NAME)
#TRAIN_FTS_SFM = FTS_NAME

Adding fts_mra_tempmz
Adding fts_mz_maxabun
Saving fts_mra_tempmz_maxabun_XGB_opt_trvl_SFM_COLS.txt


In [12]:
# ===== TRAIN =====
split_type = 'trvl'

# Initialize the feature selection class
smf = feature_selection.SelectModelFeatures(
    base_sfm_features_name=TRAIN_FTS_SFM,
    base_fitted_model_name=None,
    target_labels_list=target_labels_list,
    new_features_file_name=NEW_FEATURES,
    fitted_model_name=MODEL_NAME,
    fitted_model_algo=MODEL_ALGO,
    X_tr=X_trvl,
    X_vlte=X_vlte,
    split_type=split_type,
    train_labels=trvl_labels,
    valid_files=valid_files,
    valid_labels=valid_labels)

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features(file_name=FTS_NAME)
else: 
    TRAIN_FTS_DICT = None

# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_trvl,
    df_labels=trvl_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    base_model_name=BASE_MODEL_FTS + '_' + MODEL_ALGO + '_' + split_type + '_sfm',
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

Loading feature column names
Reading fts_mra_tempmz_maxabun_XGB_opt_trvl_SFM_COLS.txt
CV training ....
Basel model CVloss: ../models/fts_mra_tempmz_XGB_opt_trvl_sfm_cvloss.csv
basalt: LogLoss=0.15847 -> -0.00183
carbonate: LogLoss=0.0845 -> -0.00498
chloride: LogLoss=0.17325 -> 0.00225
iron_oxide: LogLoss=0.20251 -> -0.00854
oxalate: LogLoss=0.01159 -> -0.00251
oxychlorine: LogLoss=0.15008 -> -0.00967
phyllosilicate: LogLoss=0.2372 -> -0.0021
silicate: LogLoss=0.18702 -> 0.00028
sulfate: LogLoss=0.16424 -> -0.00932
sulfide: LogLoss=0.0811 -> -0.00057
Full training .....
basalt - nfeatures: 1646
carbonate - nfeatures: 287
chloride - nfeatures: 1667
iron_oxide - nfeatures: 1678
oxalate - nfeatures: 34
oxychlorine - nfeatures: 1676
phyllosilicate - nfeatures: 1672
silicate - nfeatures: 1683
sulfate - nfeatures: 1670
sulfide - nfeatures: 1683
Saving fts_mra_tempmz_maxabun_XGB_opt_trvl_COLS_sfm.txt
CV LogLoss: 0.14499
VAL LogLoss: 0.01087


**SAM TRAINING**